# Testing custom models

## Setup and imports

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My Drive/DeepLearningX/gitclone/light-weight-refinenet/src

/content/drive/My Drive/DeepLearningX/gitclone/light-weight-refinenet/src


In [0]:
# general libs
import argparse
import logging
import os
import random
import re
import sys
import time

# misc
import cv2
import numpy as np

# pytorch libs
import torch
import torch.nn as nn
from sklearn.metrics import confusion_matrix

# custom libs
from util import *


## Methods for testing

In [0]:
# defining methods for testing
sys.path.append("..")

from models.mobilenet import mbv2
from models.resnet import rf_lw50, rf_lw101, rf_lw152

from torchvision import transforms
from torch.utils.data import DataLoader, random_split
# Custom libraries
from datasets import NYUDataset as Dataset
from datasets import Pad, RandomCrop, RandomMirror, ResizeShorterScale, ToTensor, Normalise

def create_loaders(
    val_dir, val_list,
    shorter_side, crop_size, low_scale, high_scale,
    normalise_params, batch_size, num_workers, ignore_label
    ):
    """
    Args:
      train_dir (str) : path to the root directory of the training set.
      val_dir (str) : path to the root directory of the validation set.
      train_list (str) : path to the training list.
      val_list (str) : path to the validation list.
      shorter_side (int) : parameter of the shorter_side resize transformation.
      crop_size (int) : square crop to apply during the training.
      low_scale (float) : lowest scale ratio for augmentations.
      high_scale (float) : highest scale ratio for augmentations.
      normalise_params (list / tuple) : img_scale, img_mean, img_std.
      batch_size (int) : training batch size.
      num_workers (int) : number of workers to parallelise data loading operations.
      ignore_label (int) : label to pad segmentation masks with

    Returns:
      train_loader, val loader

    """
    composed_val = transforms.Compose([Normalise(*normalise_params),
                                    ToTensor()])
    valset = Dataset(data_file=val_list,
                         data_dir=val_dir,
                         transform_trn=None,
                         transform_val=composed_val)
    logger.info(" Created val set = {} examples"
                .format(len(valset)))
    ## Training and validation loaders ##

    val_loader = DataLoader(valset,
                            batch_size=1,
                            shuffle=False,
                            num_workers=num_workers,
                            pin_memory=True)
    return val_loader

def compute_iu(cm):
    """Compute IU from confusion matrix.

    Args:
      cm (Tensor) : square confusion matrix.

    Returns:
      IU vector (Tensor).

    """
    pi = 0
    gi = 0
    ii = 0
    denom = 0
    n_classes = cm.shape[0]
    IU = np.ones(n_classes)
    
    for i in range(n_classes):
        pi = sum(cm[:, i])
        gi = sum(cm[i, :])
        ii = cm[i, i]
        denom = pi + gi - ii
        if denom > 0:
            IU[i] = ii / denom
    return IU

def validate(
    segmenter, val_loader, epoch, path, num_classes=-1
    ):
    """Validate segmenter

    Args:
      segmenter (nn.Module) : segmentation network
      val_loader (DataLoader) : training data iterator
      epoch (int) : current epoch
      num_classes (int) : number of classes to consider

    Returns:
      Mean IoU (float)
    """
    val_loader.dataset.set_stage('val')
    segmenter.eval()
    cm = np.zeros((num_classes, num_classes), dtype=int)
    cmList = []
    times = []
    with torch.no_grad():
        for i, sample in enumerate(val_loader):
            start = time.time()
            input = sample['image']
            target = sample['mask']
            input_var = torch.autograd.Variable(input).float().cuda()
            # Compute output
            start = time.time()
            output = segmenter(input_var)
            end = time.time()
            times.append((end-start)*1.)
            output = cv2.resize(output[0, :num_classes].data.cpu().numpy().transpose(1, 2, 0),
                                target.size()[1:][::-1],
                                interpolation=cv2.INTER_CUBIC).argmax(axis=2).astype(np.uint8)
            # Compute IoU
            gt = target[0].data.cpu().numpy().astype(np.uint8)
            gt_idx = gt < num_classes # Ignore every class index larger than the number of classes
            cm += confusion_matrix(output[gt_idx], gt[gt_idx])
            cmList.append(np.mean( compute_iu( confusion_matrix(output[gt_idx], gt[gt_idx]) ) ))
            
            # if i % PRINT_EVERY == 0:
            #     logger.info(' Val epoch: {} [{}/{}]\t'
            #                 'Mean IoU: {:.3f}'.format(
            #                     epoch, i, len(val_loader),
            #                     compute_iu(cm).mean()
            #                 ))

    ious = compute_iu(cm)
    logger.info(" IoUs: {}".format(ious))
    miou = np.mean(ious)
    
    with open(path, 'a') as f:
      f.write("{}\n".format(miou))

    logger.info(' Val epoch: {}\tMean IoU: {:.3f}'.format(
                                epoch, miou))
    return miou, cmList, times

## Loading models, configurations and testing

In [0]:
mob = torch.load("/content/drive/My Drive/DeepLearningX/models/MobileNet/model_44")
mobFreeze = torch.load("/content/drive/My Drive/DeepLearningX/models/MobileNet/modelFreeze_17")

rest50 = torch.load("/content/drive/My Drive/DeepLearningX/models/ResNet/model_res50_unfreezed_23")
rest50Freeze = torch.load("/content/drive/My Drive/DeepLearningX/models/ResNet/model_res50_freezed_31")

rest101 = torch.load("/content/drive/My Drive/DeepLearningX/models/ResNet/model_res101_unfreezed_6")
rest101Freeze = torch.load("/content/drive/My Drive/DeepLearningX/models/ResNet/model_res101_freezed_31")

rest152 = torch.load("/content/drive/My Drive/DeepLearningX/models/ResNet/model_res152_unfreezed_54")
rest152Freeze = torch.load("/content/drive/My Drive/DeepLearningX/models/ResNet/model_res152_freezed_82")

In [0]:
models = [mob, mobFreeze, rest50, rest50Freeze, rest101, rest101Freeze, rest152, rest152Freeze]

In [0]:
VAL_DIR = "/content/drive/My Drive/DeepLearningX/TrainData-People/Test/"
VAL_LIST = "/content/drive/My Drive/DeepLearningX/TrainData-People/Test/Test.txt"
metricPath = '/content/drive/My Drive/DeepLearningX/models/'

SHORTER_SIDE = 350
CROP_SIZE = 500
NORMALISE_PARAMS = [1./255, # SCALE
                    np.array([0.485, 0.456, 0.406]).reshape((1, 1, 3)), # MEAN
                    np.array([0.229, 0.224, 0.225]).reshape((1, 1, 3))] # STD
BATCH_SIZE = 10
NUM_WORKERS = 16
NUM_CLASSES = 21
LOW_SCALE = 0.5
HIGH_SCALE = 2.0
IGNORE_LABEL = 255



In [0]:
logging.basicConfig(level=logging.INFO)
global logger 
logger = logging.getLogger(__name__)

mious = []
ious = []
times = []
tppList =[]

for model in models:

  ## Generate Segmenter ##
  segmenter = model

  val_loader = create_loaders(VAL_DIR,
                              VAL_LIST,
                              SHORTER_SIDE,
                              CROP_SIZE,
                              LOW_SCALE,
                              HIGH_SCALE,
                              NORMALISE_PARAMS,
                              BATCH_SIZE,
                              NUM_WORKERS,
                              IGNORE_LABEL)

  start = time.time()
  miou, cmList, tpp = validate(segmenter, val_loader, 0, metricPath + "mious_test.txt", 2)
  end = time.time()
  logger.info("Stage  finished, time spent {:.3f}min, avg time per picture {:.5f}sec".format(
            (end - start) / 60., np.mean(tpp)))
  mious.append(miou)
  ious.append(cmList)
  times.append((end-start) / 60.)
  tppList.append(tpp)

INFO:__main__: Created val set = 104 examples
INFO:__main__: IoUs: [0.90915827 0.70786982]
INFO:__main__: Val epoch: 0	Mean IoU: 0.809
INFO:__main__:Stage  finished, time spent 0.760min, avg time per picture 0.01562sec
INFO:__main__: Created val set = 104 examples
INFO:__main__: IoUs: [0.90958184 0.70624557]
INFO:__main__: Val epoch: 0	Mean IoU: 0.808
INFO:__main__:Stage  finished, time spent 0.598min, avg time per picture 0.02273sec
INFO:__main__: Created val set = 104 examples
INFO:__main__: IoUs: [0.91460936 0.71157932]
INFO:__main__: Val epoch: 0	Mean IoU: 0.813
INFO:__main__:Stage  finished, time spent 0.656min, avg time per picture 0.02400sec
INFO:__main__: Created val set = 104 examples
INFO:__main__: IoUs: [0.91836034 0.71161172]
INFO:__main__: Val epoch: 0	Mean IoU: 0.815
INFO:__main__:Stage  finished, time spent 0.653min, avg time per picture 0.02401sec
INFO:__main__: Created val set = 104 examples
INFO:__main__: IoUs: [0.89231874 0.66038141]
INFO:__main__: Val epoch: 0	Mean 